In [1]:
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from string import punctuation
from pymystem3 import Mystem
from natasha import DatesExtractor, MoneyExtractor
from extractor import NumberExtractor
from NER_filters import (NER_filter_percents, NER_filter_time_periods,
                         NER_monetary_amounts, NER_filter_month_year, NER_filter_currency)


tokenizer = RegexpTokenizer(r"[а-яёa-z0-9]+")

stemmer_mystem = Mystem()
datesextractor = DatesExtractor()
moneyextractor = MoneyExtractor()
numberextractor = NumberExtractor()


#Типы продуктов и вопросы для уточнения:
product_types_info = pd.read_csv('product_types.csv', sep=';')
# ключевые описания продуктов:
products_info = pd.read_csv('products.csv', sep=';')
product_types = product_types_info['product_type'].values
product_types = product_types[:-1] #  кроме not recognized

specify_questions_list = product_types_info.values
specify_questions = {product[0]:product[1] for product in specify_questions_list}


# #Пример входного словаря d:

# d = dict({'text':'Хочу узнать статус черной карты с котиком',
#          'Evolution':['01.01.2021'],
#          'Premium':['01.01.2021'],
#          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
#          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл']})  

# #Запуск:
# trigger(d)   

# #Возвращает тот же словарь с доп полем target:
# {'text': 'Хочу узнать статус черной карты с котиком',
#  'Evolution': ['01.01.2021'],
#  'Premium': ['01.01.2021'],
#  'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
#  'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
#  'Ипотека на Дом и земельный участок': ['5000000 долл',
#   '180 мес',
#   '10000 долл'],
#  'target': 'Evolution'}


# #Пример словаря с полным перечнем продуктов
# d = dict({'text':'Хочу узнать статус черной карты с котиком',
#          'Aurum':['01.01.2021'],
#          'Generation':['01.01.2021'],
#          'Evolution':['01.01.2021'],
#          'Premium':['01.01.2021'],
#          'Mir':['01.01.2021'],
#          'Emotion':['01.01.2021'],
#          'Кредит наличными':['5000000 долл', '180 мес', '10000 долл'],
#          'Кредит под залог недвижимости':['5000000 евро', '180 мес', '10000 евро'],
#          'Кредит (Рефинансирование)':['5000000 руб', '180 мес', '10000 руб'],
#          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
#          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Новостройки с господдержкой':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Новостройки':['5000000 руб', '180 мес', '10000 руб'],
#          'Рефинансирование ипотеки':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека Господдержка для семей с детьми':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Коммерческую недвижимость':['5000000 руб', '180 мес', '10000 руб'],
#          'Ипотека на Дом и земельный участок':['5000000 руб', '180 мес', '10000 руб']})

#Формат в точности как в примере: 
# даты через точки, 
# сроки в месяцах (через "мес"), 
# валюты можно как в кратком (р, руб, долл), так и в полном формате (рублей, долларов)


def filter_text(text, tokenizer = tokenizer, stemmer_mystem = stemmer_mystem):
    text = str(text)
    tokens = tokenizer.tokenize(text.lower())
    tokens = stemmer_mystem.lemmatize(" ".join(tokens))
    tokens = [token for token in tokens if token != " "
              and token.strip() not in punctuation]
    return " ".join(tokens)


def find_product_by_description(d, lst_cards,lst_indexes,products_names,specify_question):
#если дата открытия вклада не указана (или не удалось отсканировать) - ищем по описаниям
    dd=[]
    if len(lst_indexes) == 1:#если нашелся один продукт по описанию
        product_name = products_names[lst_indexes[0]]#определяем название
        if product_name in d:#если такой продукт есть у клиента
            d['target'] = product_name
            return d
        else:
            d['target']=specify_question #запрашиваем доп инфу
            return d

    elif len(lst_indexes)>1: #если нашлось несколько продуктов по описанию
        lsss = [len(i) for i in lst_cards] #определяем число ключевых слов для каждого продукта
        card_indexes = [i for i, j in enumerate(lsss) if j == max(lsss)] #определяем продукт с макс числом ключевых слов
        if len(card_indexes)==1:#если макс только у одной карты

            product_name = products_names[card_indexes[0]]#определяем ее название
            if product_name in d:#если такая карта есть у клиента
                d['target'] = product_name#выдаем в таргет эту карту
                return d
            else:#если такой карты нет у клиента
                d['target']=specify_question#запрашиваем доп инфу
            return d
        elif len(card_indexes)>1:#если макс у нескольких карт
            whc =[j for j in [products_names[i] for i in card_indexes] if j in d]#определяем какая из карт есть у клиента
            if len(whc)==1:#если у клиента есть одна такая карта
                d['target'] = whc#выдаем в таргет эту карту
                return d
            else:#если у клиента несколько таких карт или таких карт нет
                d['target']=specify_question#запрашиваем доп инфу
                return d
    else:
        return dd



def recognize_credit_parameters(d, dats, product_names, specify_question):
    par1=[]
    par1_names=[]
    for j,i in enumerate(d.values()):#сканируем значения словаря
        if list(d.keys())[j] in product_names:#если найденный продукт в списке продуктов
            par2=[]
            for ii in i:
                #сканируем суммы
                matches = moneyextractor(ii)
                for match in matches:
                    par2.append([match.fact.as_json['integer'],match.fact.as_json['currency']])
                #сканируем сроки
                textn = filter_text(ii)
                lsst = textn.split()
                ind_year = [ij for ij,s in enumerate(lsst) if s.isdigit()]
                for ij in ind_year:
                    try:
                        if lsst[ij+1]=='мес':
                            if len(lsst[ij])<=3:
                                yearn = int(lsst[ij])
                                par2.append(yearn)
                                par1_names.append(list(d.keys())[j])
                                break
                    except IndexError:
                        continue
            par1.append(par2)

    par1 = list(filter(None, par1)) #итоговый список кредитов клиента
    #par1  =[[[200000, 'RUB'], 150, [10000, 'RUB']], [[200000, 'RUB'], 50, [10000, 'RUB']]]
    print('Client products in dict recognized:',list(zip(par1,par1_names)))
    if any(par1):#если у клиента есть кредиты
        par2=[]#сопоставляем с кредитами, найденными в тексте
        for w,k in enumerate(par1):
            par3=[]
            for i in k:
                for j in dats:
                    try:
                        j=j[0]
                        if j==i:
                            par3.append(w)
                    except:
                        if j==i:
                            par3.append(w)
            par2.append(len(par3))#сколько совпадений по каждому кредиту
        #par2 = [3, 2]
        print('Lengths of common parameters:',list(zip(par2,par1_names)))
        inss = [i for i, j in enumerate(par2) if j == max(par2)] #определяем кредит с макс числом совпадений
        #inss = [0]

        if len(inss)==1:
            d['target'] = par1_names[inss[0]]#выдаем в таргет этот кредит
            return d
        elif len(inss)>1:#если макс у нескольких кредитов
            d['target']=specify_question#запрашиваем доп инфу
            return d
    else:#если не удалось
        d['target']=specify_questions['not recognized']#запрашиваем доп инфу
        return d



def recognize_card_parameters(d, dats, product_names, specify_question):
    #сверяем со сроками всех карт клиента
    par2=[]
    for j,i in enumerate(d.values()):#сканируем значения словаря
        matches = datesextractor(i[0])
        par1=[]
        for match in matches:
            par1.append([match.fact.as_json['year'],match.fact.as_json['month']])
        if par1:#если удалось распознать даты карт клиента (если они есть)
            if len(par1[0]) == len(dats):#если есть совпадение по срокам
                if par1[0] == dats:
                    par2.append(j)
            else:
                if par1[0][0] == dats[0]:
                    par2.append(j)
    if par2:
        if len(par2)==1:#если есть одно совпадение
            d['target']=list(d.keys())[par2[0]] #нашлась карта с таким же сроком и она одна - выдаем ее в таргет
            return d
        else:#если совпадений несколько
            d['target']=specify_question#запрашиваем доп инфу
            return d
    else:#если не удалось
        d['target']=specify_question#запрашиваем доп инфу
        return d



def recognize_deposit_parameters(d, dats, product_names, specify_question):
    #поиск кредитов клиента
    par1=[]
    par1_names=[]
    for j,i in enumerate(d.values()):#сканируем значения словаря
        if list(d.keys())[j] in product_names:#если найденный продукт в списке продуктов
            par2=[]
            for ii in i:
                #поиск остатка на счете их валюты
                matches = moneyextractor(ii)
                for match in matches:
                    par2.append([match.fact.as_json['integer'],match.fact.as_json['currency']])
                percents = NER_filter_percents(ii)
                for percent in percents:
                    par2.append(percent)
                par1_names.append(list(d.keys())[j])
            par1.append(par2)

    par1 = list(filter(None, par1)) #итоговый список вкладов клиента
    #print(par1)
    print('Client products in dict recognized:',list(zip(par1,par1_names)))
    if any(par1):#если у клиента есть вклады
        par2=[]#сопоставляем с вкладами, найденными в тексте
        for w,k in enumerate(par1):
            par3=[]
            for i in k:
                for j in dats:
                    try:
                        j=j[0]
                        if j==i:
                            par3.append(w)
                    except:
                        if j==i:
                            par3.append(w)
            par2.append(len(par3))#сколько совпадений по каждому вкладу
        #par2 = [3, 2]
        print('Lengths of common parameters:',list(zip(par2,par1_names)))
        inss = [i for i, j in enumerate(par2) if j == max(par2)] #определяем вклад с макс числом совпадений
        #inss = [0]

        if len(inss)==1:
            d['target'] = par1_names[inss[0]]#выдаем в таргет этот вклад
            return d
        elif len(inss)>1:#если макс у нескольких вкладов
            d['target']=specify_question#запрашиваем доп инфу
            return d

    else:#если не удалось
        d['target']=specify_questions['not recognized']#запрашиваем доп инфу
        return d



def recognize_account_parameters(d, dats, product_names, specify_question):
    #поиск счетов клиента
    par1=[]
    par1_names=[]
    for j,i in enumerate(d.values()):#сканируем значения словаря
        if list(d.keys())[j] in product_names:#если найденный продукт в списке продуктов
            par2=[]
            for ii in i:
                #поиск остатка на счете их валюты
                matches = moneyextractor(ii)
                for match in matches:
                    par2.append([match.fact.as_json['integer'],match.fact.as_json['currency']])
                #дата открытия счета
                dates = datesextractor(ii)
                for date in dates:
                    par2.append([date.fact.as_json['year'],date.fact.as_json['month']])
                #валюта
                textn = filter_text(ii)
                lsst = textn.split()
                for l in lsst:
                    if l in ['рубль','доллар','евро']:
                        currency = l
                        par2.append(currency)
                        par1_names.append(list(d.keys())[j])
                        break
            par1.append(par2)

    par1 = list(filter(None, par1)) #итоговый список счетов клиента
    #print(par1)
    print('Client products in dict recognized:',list(zip(par1,par1_names)))
    if any(par1):#если у клиента есть счета
        par2=[]#сопоставляем с счетами, найденными в тексте
        for w,k in enumerate(par1):
            par3=[]
            for i in k:
                for j in dats:
                    try:
                        j=j[0]
                        if j==i:
                            par3.append(w)
                    except:
                        if j==i:
                            par3.append(w)
            par2.append(len(par3))#сколько совпадений по каждому счету
        #par2 = [3, 2]
        print('Lengths of common parameters:',list(zip(par2,par1_names)))
        inss = [i for i, j in enumerate(par2) if j == max(par2)] #определяем счет с макс числом совпадений
        #inss = [0]

        if len(inss)==1:
            d['target'] = par1_names[inss[0]]#выдаем в таргет этот счет
            return d
        elif len(inss)>1:#если макс у нескольких кредитов
            d['target']=specify_question#запрашиваем доп инфу
            return d

    else:#если не удалось
        d['target']=specify_questions['not recognized']#запрашиваем доп инфу
        return d



def find_by_product_parameters(d,tokens,product_type,products_names,specify_question):
    message_text = d['text']
    if product_type == 'карта':
        dats = NER_filter_month_year(message_text, tokens, datesextractor)
        if any(dats):
            dd = recognize_card_parameters(d, dats,products_names, specify_question)
            if dd:
                return dd
        else:#если не удалось
            return None
    elif product_type == 'кредит' or product_type == 'ипотека':
        credit_sum = NER_monetary_amounts(message_text, tokens, moneyextractor)
        credit_term = NER_filter_time_periods(tokens)
        dats = [credit_sum, credit_term]
        if any(dats):
            dd = recognize_credit_parameters(d, dats,products_names, specify_question)
            if dd:
                return dd
        else:#если не удалось
            return None
    elif product_type == 'вклад':
        percents = NER_filter_percents(tokens)
        deposits_money = NER_monetary_amounts(message_text, tokens, moneyextractor)
        dats = [percents, deposits_money]
        if any(dats):
            dd = recognize_deposit_parameters(d, dats,products_names, specify_question)
            if dd:
                return dd
        else:#если не удалось
            return None
    elif product_type == 'счет':
        account_money = NER_monetary_amounts(message_text, tokens, moneyextractor)
        opening_date = NER_filter_month_year(message_text, tokens, datesextractor)
        currency = NER_filter_currency(tokens)
        dats = [account_money, opening_date, currency]
        if any(dats):
            dd = recognize_account_parameters(d, dats,products_names, specify_question)
            if dd:
                return dd
        else:#если не удалось
            return None



def trigger(d):
    if 'text' in d:
        text = filter_text(d['text'])
        textn = text.split()
        #поиск по прямому вхождению
        for product_type in product_types:
            products = products_info[products_info['product_type'] == product_type]
            for product in products.itertuples():
                other_names = product.other_names.split(',')
                if [other_name for other_name in other_names if other_name in text]:
                    if product.product_name in d:
                        d['target'] = product.product_name
                        return d
                    else:
                        d['target'] = specify_questions[product_type]
                        return d
        #распознавание сущностей и соотнесение с параметрами продукта
        for product_type in product_types:
            products = products_info[products_info['product_type'] == product_type]
            products_names = products['product_name'].values
            dd = find_by_product_parameters(d,textn,product_type,products_names,specify_questions[product_type])
            if dd:
                return dd
        #по ключевым словам
        for product_type in product_types:
            products = products_info[products_info['product_type'] == product_type]
            keywords_by_product = []
            for product in products.itertuples():
                keywords = product.keywords.split(',')
                keywords_in_text = [keyword for keyword in keywords if keyword in textn]
                keywords_by_product.append(keywords_in_text)
            lst_indexes = [product_index for product_index, encountered_keywords in enumerate(keywords_by_product) if encountered_keywords]
            if lst_indexes:
                products_names = products['product_name'].values
                dd = find_product_by_description(d,keywords_by_product,lst_indexes,products_names,specify_questions[product_type])
                if dd:
                    return dd
        d['target']=specify_questions['not recognized']
        return d
    else:
        return []

## Тесты:

In [2]:
d = dict({'text':'Хочу узнать про свою ипотеку',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '170 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '170 мес', '10000 долл']})
trigger(d)

{'text': 'Хочу узнать про свою ипотеку',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '170 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '170 мес',
  '10000 долл'],
 'target': 'Уточните, по какому ипотечному кредиту Вы хотите получить информацию?'}

In [3]:
d = dict({'text':'Не отображается сумма по вкладу',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл'],
          'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
          }) 
trigger(d)

{'text': 'Не отображается сумма по вкладу',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '180 мес',
  '10000 долл'],
 'Просто накопить': ['5000000 руб', '3.4%', '01.01.201412.01.2021'],
 'target': ['Просто накопить']}

In [4]:
d = dict({'text':'Почему кэшбек по моей карте, где нарисован леопард такой маленький?',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл'],
          'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
          })
trigger(d)

{'text': 'Почему кэшбек по моей карте, где нарисован леопард такой маленький?',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '180 мес',
  '10000 долл'],
 'Просто накопить': ['5000000 руб', '3.4%', '01.01.201412.01.2021'],
 'target': ['Evolution']}

In [5]:
d = dict({'text':'Как закрыть вклад?',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл'],
          'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
          })
trigger(d)

{'text': 'Как закрыть вклад?',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '180 мес',
  '10000 долл'],
 'Просто накопить': ['5000000 руб', '3.4%', '01.01.201412.01.2021'],
 'target': ['Просто накопить']}

In [6]:
d = dict({'text':'Начисления по вкладу уверенное будущее ниже чем должно быть.',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл'],
          'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
          'Уверенное будущее':['1000000 руб', '2.4%','01.01.2015' '12.01.2022']
          })
trigger(d)

{'text': 'Начисления по вкладу уверенное будущее ниже чем должно быть.',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '180 мес',
  '10000 долл'],
 'Просто накопить': ['5000000 руб', '3.4%', '01.01.201412.01.2021'],
 'Уверенное будущее': ['1000000 руб', '2.4%', '01.01.201512.01.2022'],
 'target': 'Уверенное будущее'}

In [7]:
d = dict({'text':'Начисления по вкладу там где 1 млн ниже чем должно быть.',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл'],
          'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
          'Уверенное будущее':['1000000 руб', '2.4%','01.01.2015' '12.01.2022']
          })
trigger(d)

{'text': 'Начисления по вкладу там где 1 млн ниже чем должно быть.',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '180 мес',
  '10000 долл'],
 'Просто накопить': ['5000000 руб', '3.4%', '01.01.201412.01.2021'],
 'Уверенное будущее': ['1000000 руб', '2.4%', '01.01.201512.01.2022'],
 'target': 'Уточните, по какому вкладу Вы хотите получить информацию?'}

In [8]:
d = dict({'text':'Вклад на 3.4 процента',
          'Evolution':['01.01.2021'],
          'Premium':['01.01.2021'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '180 мес', '10000 долл'],
          'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
          '367 дней Я сам!':['1000000 руб', '2.4%','01.01.2015' '12.01.2022']
          })
trigger(d)

Client products in dict recognized: [([[5000000, 'RUB'], '4'], 'Просто накопить'), ([[1000000, 'RUB'], '4'], 'Просто накопить')]
Lengths of common parameters: [(1, 'Просто накопить'), (1, 'Просто накопить')]


{'text': 'Вклад на 3.4 процента',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.01.2021'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '180 мес', '10000 р'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '180 мес',
  '10000 долл'],
 'Просто накопить': ['5000000 руб', '3.4%', '01.01.201412.01.2021'],
 '367 дней Я сам!': ['1000000 руб', '2.4%', '01.01.201512.01.2022'],
 'target': 'Уточните, по какому вкладу Вы хотите получить информацию?'}

In [9]:
d = dict({'text':'счет с остатком 50$',
          'Evolution':['01.01.2021'],
          'Premium':['01.06.2020'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '18 мес', '10005.56 р'],
          'Кредит наличными':['2000 долл', '10 мес', '100 долл'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '18 мес', '10000 долл'],
          'Текущий счет Зарплата': ['01.01.2020','10000 руб', 'рубль'],
         'Текущий счет Переводы и платежи': ['05.05.2020','50 долл', 'доллар'] })
trigger(d)

Client products in dict recognized: [([[5000000, 'RUB'], 18, [10005, 'RUB']], 'Кредит на личное подсобное хозяйство'), ([[2000, 'USD'], 10, [100, 'USD']], 'Кредит наличными')]
Lengths of common parameters: [(0, 'Кредит на личное подсобное хозяйство'), (0, 'Кредит наличными')]


{'text': 'счет с остатком 50$',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.06.2020'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '18 мес', '10005.56 р'],
 'Кредит наличными': ['2000 долл', '10 мес', '100 долл'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '18 мес',
  '10000 долл'],
 'Текущий счет Зарплата': ['01.01.2020', '10000 руб', 'рубль'],
 'Текущий счет Переводы и платежи': ['05.05.2020', '50 долл', 'доллар'],
 'target': 'Уточните, по какому кредиту Вы хотите получить информацию?'}

In [10]:
d = dict({'text':'счет открытый в мае этого года',
          'Evolution':['01.01.2021'],
          'Premium':['01.06.2020'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '18 мес', '10005.56 р'],
          'Кредит наличными':['2000 долл', '10 мес', '100 долл'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '18 мес', '10000 долл'],
          'Текущий счет Зарплата': ['01.01.2020','10000 руб', 'рубль'],
          'Текущий счет Переводы и платежи': ['05.05.2020','50 долл', 'доллар'] })

trigger(d)

{'text': 'счет открытый в мае этого года',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.06.2020'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '18 мес', '10005.56 р'],
 'Кредит наличными': ['2000 долл', '10 мес', '100 долл'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '18 мес',
  '10000 долл'],
 'Текущий счет Зарплата': ['01.01.2020', '10000 руб', 'рубль'],
 'Текущий счет Переводы и платежи': ['05.05.2020', '50 долл', 'доллар'],
 'target': 'Текущий счет Переводы и платежи'}

In [11]:
d = dict({'text':'счет открытый в январе прошлого года',
          'Evolution':['01.01.2021'],
          'Premium':['01.06.2020'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '18 мес', '10005.56 р'],
          'Кредит наличными':['2000 долл', '10 мес', '100 долл'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '18 мес', '10000 долл'],
          'Текущий счет Зарплата': ['01.01.2019','10000 руб', 'рубль'],
          'Текущий счет Переводы и платежи': ['05.05.2020','50 долл', 'доллар'] })
trigger(d)

{'text': 'счет открытый в январе прошлого года',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.06.2020'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '18 мес', '10005.56 р'],
 'Кредит наличными': ['2000 долл', '10 мес', '100 долл'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '18 мес',
  '10000 долл'],
 'Текущий счет Зарплата': ['01.01.2019', '10000 руб', 'рубль'],
 'Текущий счет Переводы и платежи': ['05.05.2020', '50 долл', 'доллар'],
 'target': 'Текущий счет Зарплата'}

In [12]:
d = dict({'text':'счет в рублях',
          'Evolution':['01.01.2021'],
          'Premium':['01.06.2020'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '18 мес', '10005.56 р'],
          'Кредит наличными':['2000 долл', '10 мес', '100 долл'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '18 мес', '10000 долл'],
          'Текущий счет Зарплата': ['01.01.2019','10000 руб', 'рубль'],
          'Текущий счет Переводы и платежи': ['05.05.2020','50 долл', 'доллар'] })
trigger(d)

Client products in dict recognized: [([[2019, 1], [10000, 'RUB'], 'рубль'], 'Текущий счет Зарплата'), ([[2020, 5], [50, 'USD'], 'доллар'], 'Текущий счет Переводы и платежи')]
Lengths of common parameters: [(0, 'Текущий счет Зарплата'), (0, 'Текущий счет Переводы и платежи')]


{'text': 'счет в рублях',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.06.2020'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '18 мес', '10005.56 р'],
 'Кредит наличными': ['2000 долл', '10 мес', '100 долл'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '18 мес',
  '10000 долл'],
 'Текущий счет Зарплата': ['01.01.2019', '10000 руб', 'рубль'],
 'Текущий счет Переводы и платежи': ['05.05.2020', '50 долл', 'доллар'],
 'target': 'Уточните, по какому текущему счету Вы хотите получить информацию?'}

In [13]:
d = dict({'text':'карта до июня',
          'Evolution':['01.01.2021'],
          'Premium':['01.06.2020'],
          'Кредит на личное подсобное хозяйство':['5000000 р', '18 мес', '10005.56 р'],
          'Кредит наличными':['2000 долл', '10 мес', '100 долл'],
          'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
          'Ипотека на Дом и земельный участок':['5000000 долл', '18 мес', '10000 долл'],
          'Текущий счет Зарплата': ['01.01.2019','10000 руб', 'рубль'],
          'Текущий счет Переводы и платежи': ['05.05.2020','50 долл', 'доллар'] })
trigger(d)

{'text': 'карта до июня',
 'Evolution': ['01.01.2021'],
 'Premium': ['01.06.2020'],
 'Кредит на личное подсобное хозяйство': ['5000000 р', '18 мес', '10005.56 р'],
 'Кредит наличными': ['2000 долл', '10 мес', '100 долл'],
 'Ипотека на Вторичное жилье': ['5000000 руб', '180 мес', '10000 руб'],
 'Ипотека на Дом и земельный участок': ['5000000 долл',
  '18 мес',
  '10000 долл'],
 'Текущий счет Зарплата': ['01.01.2019', '10000 руб', 'рубль'],
 'Текущий счет Переводы и платежи': ['05.05.2020', '50 долл', 'доллар'],
 'target': 'Уточните, по какой карте Вы хотите получить информацию?'}

## Вычисление accuracy

In [2]:
test_data = pd.read_csv('test_2.csv', sep=';')

test_data['predict']=0

for i in range(len(test_data)):
    #предполагаем, что у клиента есть все продукты
    d = dict({'text':test_data['text'][i],
              'Aurum':['01.01.2021'],
             'Generation':['01.01.2021'],
             'Evolution':['01.01.2021'],
             'Premium':['01.01.2021'],
             'Mir':['01.01.2021'],
             'Emotion':['01.01.2021'],
             'Кредит наличными':['5000000 долл', '180 мес', '10000 долл'],
             'Кредит под залог недвижимости':['5000000 евро', '180 мес', '10000 евро'],
             'Кредит (Рефинансирование)':['5000000 руб', '180 мес', '10000 руб'],
             'Кредит на личное подсобное хозяйство':['5000000 р', '180 мес', '10000 р'],
             'Ипотека на Вторичное жилье':['5000000 руб', '180 мес', '10000 руб'],
             'Ипотека на Новостройки с господдержкой':['5000000 руб', '180 мес', '10000 руб'],
             'Ипотека на Новостройки':['5000000 руб', '180 мес', '10000 руб'],
             'Рефинансирование ипотеки':['5000000 руб', '180 мес', '10000 руб'],
             'Ипотека Господдержка для семей с детьми':['5000000 руб', '180 мес', '10000 руб'],
             'Ипотека на Коммерческую недвижимость':['5000000 руб', '180 мес', '10000 руб'],
             'Ипотека на Дом и земельный участок':['5000000 руб', '180 мес', '10000 руб'],
            'Текущий счет Зарплата': ['01.01.2019','10000 руб', 'рубль'],
            'Текущий счет Переводы и платежи': ['05.05.2020','50 долл', 'доллар'],
              'Просто накопить':['5000000 руб', '3.4%','01.01.2014' '12.01.2021'],
            '367 дней Я сам!':['1000000 руб', '2.4%','01.01.2015' '12.01.2022'],
              'Просто преумножить':['1000000 руб', '2.4%','01.01.2015' '12.01.2022'],
              'Просто управлять':['1000000 руб', '2.4%','01.01.2015' '12.01.2022'],
              'Просто поймать момент':['1000000 руб', '2.4%','01.01.2015' '12.01.2022'],
              'Уверенное будущее':['1000000 руб', '2.4%','01.01.2015' '12.01.2022']})  
    
    test_data['predict'][i] = trigger(d)['target']  

C:\Users\darkj\anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\darkj\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Client products in dict recognized: [([[5000000, 'USD'], 180, [10000, 'USD']], 'Кредит наличными'), ([[5000000, 'EUR'], 180, [10000, 'EUR']], 'Кредит под залог недвижимости'), ([[5000000, 'RUB'], 180, [10000, 'RUB']], 'Кредит (Рефинансирование)'), ([[5000000, 'RUB'], 180, [10000, 'RUB']], 'Кредит на личное подсобное хозяйство')]
Lengths of common parameters: [(0, 'Кредит наличными'), (0, 'Кредит под залог недвижимости'), (0, 'Кредит (Рефинансирование)'), (0, 'Кредит на личное подсобное хозяйство')]
Client products in dict recognized: [([[2019, 1], [10000, 'RUB'], 'рубль'], 'Текущий счет Зарплата'), ([[2020, 5], [50, 'USD'], 'доллар'], 'Текущий счет Переводы и платежи')]
Lengths of common parameters: [(0, 'Текущий счет Зарплата'), (0, 'Текущий счет Переводы и платежи')]
Client products in dict recognized: [([[5000000, 'USD'], 180, [10000, 'USD']], 'Кредит наличными'), ([[5000000, 'EUR'], 180, [10000, 'EUR']], 'Кредит под залог недвижимости'), ([[5000000, 'RUB'], 180, [10000, 'RUB']], 'К

In [3]:
test_data

,text,product,predict
0,"Здравствуйте, какой у Карты процент на остаток?","Уточните, по какой карте Вы хотите получить ин...","Уточните, по какой карте Вы хотите получить ин..."
1,"Мне дали карту 14 декабря, вопрос: с какого чи...","Уточните, по какой карте Вы хотите получить ин...","Уточните, по какой карте Вы хотите получить ин..."
2,Добрый день! На мою карту evolution был начисл...,Evolution,Evolution
3,Здравствуйте. У меня карта evolution. Забыл пи...,Evolution,Evolution
4,"Добрый день, подскажите пожалуйста как я могу ...","Уточните, по какой карте Вы хотите получить ин...","Уточните, по какому банковскому продукту Вы хо..."
...,...,...,...
100,"Доброе утро! Вопрос по вкладу ""Надёжное будуще...",Уверенное будущее,Уверенное будущее
101,почему нет возможности пополнить валютный вкла...,"Уточните, по какому вкладу Вы хотите получить ...","Уточните, по какому вкладу Вы хотите получить ..."
102,Добрый день. Не пополняется вклад,"Уточните, по какому вкладу Вы хотите получить ...","Уточните, по какому вкладу Вы хотите получить ..."
103,"Добрый день, хочу перекинуть деньги со счета ""...",Просто поймать момент,Просто поймать момент


In [4]:
#считаем точность

from sklearn.metrics import accuracy_score
accuracy_score(test_data['product'], test_data['predict'])

0.8952380952380953